#**Notebook 04**
- **Professor:** Iális Cavalcante
- **Monitor:** Iago Magalhães
- **Disciplina:** Ciência de dados
- **Curso:** Engenharia da Computação
- **Descrição:**
No notebook 04 iremos aprender sobre regressão polinomial.
- **Questão:** O aluno João Vitor deseja práticar exercícios diariamente. Usando seus conhecimentos, ele encontrou uma base de dados sobre estações de bicicleta na cidade de São Paulo e com base nas infromações contidas nela, vai buscar extrair dados para que consiga agendar as bicicletas em horários e dias com maior disponibilidade.



##Importações de bibliotecas

In [ ]:
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import pandas as pd
import numpy as np
import folium

from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import max_error

##Leitura de dados

In [ ]:
df = pd.read_csv('sp_bike.csv')
df.head()

##Análise de dados

In [ ]:
#Verificando estações ativas
df = df[df.status == 'A']
df.head()

In [ ]:
#Removendo estações duplicadas para mostrar uma de cada no mapa.
df_estacoes = df[['name', 'lat', 'lng']].drop_duplicates()
df_estacoes.info()

In [ ]:
#Gerando um gráfico com as localizações das estações na cidade de São Paulo
cord_center = [df_estacoes.lat.mean(), df_estacoes.lng.mean()]
mp = folium.Map(location=cord_center, zoom_start=13, tiles='cartodbpositron')
for i, location in df_estacoes.iterrows():
    folium.CircleMarker(
        location=[location['lat'],location['lng']],
        radius=2,
        popup=location['name']
    ).add_to(mp)
mp

###Verificar a disponibilidade das bicicletas nas estações

In [ ]:
df['created_at'] = pd.to_datetime(df.created_at)
df['hora'] = df['created_at'].apply(lambda x: dt.datetime.strftime(x, '%H'))
df['dia_semana'] = df['created_at'].dt.weekday
df.head()

In [ ]:
df_media_hora = df.groupby(['name', 'hora', 'dia_semana'], as_index=False).agg('mean')
df_media_hora['available'] = df_media_hora['available'].astype(int)
df_media_hora.head()

In [ ]:
df_media_hora[df_media_hora.name == 'Metrô Faria Lima']

In [ ]:
df_hora = df_media_hora[['hora', 'available']].groupby(['hora'], as_index=False).agg('mean')
df_hora.head()

In [ ]:
sns.lineplot(data=df_hora, x="hora", y="available")

##Algoritmo de Machine Learning

###Algoritmo de Regressão Linear

In [ ]:
X = df_hora[['hora']].astype(int).values
y = df_hora[['available']].values

model = LinearRegression()
model.fit(X, y)

y_hora_linear = model.predict(X)

In [ ]:
sns.scatterplot(data=df_hora, x='hora', y='available');
plt.plot(X, y_hora_linear, color='r')
plt.show()

In [ ]:
mean_squared_error(y, y_hora_linear, squared=False)

###Algoritmo de Regressão Polinomial

In [ ]:
polinomial = PolynomialFeatures(degree=7, include_bias=False)
X_polinomial = polinomial.fit_transform(X)

In [ ]:
model = LinearRegression()
model.fit(X_polinomial, y)

y_polinomial = model.predict(X_polinomial)

In [ ]:
sns.scatterplot(data=df_hora, x='hora', y='available');
plt.plot(X, y_polinomial, color='g')
plt.show()

In [ ]:
mean_squared_error(y, y_polinomial, squared=False)

###Treinando uma Regressão Polinomial para uma estação específica

In [ ]:
df_metro_fl = df_media_hora[df_media_hora.name == 'Metrô Faria Lima']
df_metro_fl.head()

In [ ]:
sns.lineplot(data=df_metro_fl, x="hora", y="available", hue='dia_semana')

In [ ]:
df_metro_fl_dia_semana = df_metro_fl[df_metro_fl.dia_semana < 5]

In [ ]:
sns.lineplot(data=df_metro_fl_dia_semana, x="hora", y="available", hue='dia_semana')

In [ ]:
df_metro_fl_fim_semana = df_metro_fl[df_metro_fl.dia_semana >= 5]

In [ ]:
sns.lineplot(data=df_metro_fl_fim_semana, x="hora", y="available", hue='dia_semana');

In [ ]:
df_media_dia_semana = df_metro_fl_dia_semana.groupby(['hora'], as_index=False).agg('mean')
df_media_fim_semana = df_metro_fl_fim_semana.groupby(['hora'], as_index=False).agg('mean')

In [ ]:
plt.plot(df_media_dia_semana['hora'], df_media_dia_semana['available'], color='r')
plt.plot(df_media_fim_semana['hora'], df_media_fim_semana['available'], color='b')
plt.show()

###Treinando a Regressão Polinomial para os dias da semana

In [ ]:
X_dia_semana = df_media_dia_semana[['hora']].astype(int).values
y_dia_semana = df_media_dia_semana[['available']].values

In [ ]:
polinomial = PolynomialFeatures(degree=7, include_bias=False)
X_dia_semana_polinomial = polinomial.fit_transform(X_dia_semana)

In [ ]:
model = LinearRegression()
model.fit(X_dia_semana_polinomial, y_dia_semana)

In [ ]:
y_dia_semana_polinomial = model.predict(X_dia_semana_polinomial)

In [ ]:
sns.scatterplot(data=df_media_dia_semana, x='hora', y='available');
plt.plot(X_dia_semana, y_dia_semana_polinomial, color='g')
plt.show()

In [ ]:
mean_squared_error(y_dia_semana, y_dia_semana_polinomial, squared=False)

###Treinando a Regressão Polinomial para o final de semana

In [ ]:
X_fim_semana = df_media_fim_semana[['hora']].astype(int).values
y_fim_semana = df_media_fim_semana[['available']].values

In [ ]:
polinomial = PolynomialFeatures(degree=7, include_bias=False)
X_fim_semana_polinomial = polinomial.fit_transform(X_fim_semana)

In [ ]:
model = LinearRegression()
model.fit(X_fim_semana_polinomial, y_fim_semana)

In [ ]:
y_fim_semana_polinomial = model.predict(X_fim_semana_polinomial)

In [ ]:
sns.scatterplot(data=df_media_fim_semana, x='hora', y='available');
plt.plot(X_fim_semana, y_fim_semana_polinomial, color='g')
plt.show()

In [ ]:
mean_squared_error(y_fim_semana, y_fim_semana_polinomial, squared=False)

##Atividades de casa
- Analise a possibilidade de realizar previsões de horários em que as estações tem menor movimento em dias especifícos da semana com o algoritmo de regressão polinomial.

##Referências
- [Regressão Polinomial](https://www.sakurai.dev.br/regressao-polinomial/)
- [Folium](https://python-visualization.github.io/folium/latest/)
- [Dataset Bicycle Sharing Brazil](https://www.kaggle.com/datasets/marlesson/bicycle-sharing-brazil-sp-gyn)